In [1]:
from cs336_basics import bpe_tokenizer 
from cs336_basics import Tokenizer
import pickle

In [ ]:
special_tokens = ['<|endoftext|>']
root = '../data/'
for name, vocab_size in [('TinyStoriesV2-GPT4-', 10000), ('owt_', 32000)]:
    vocab, merges = bpe_tokenizer(f'{root}{name}train.txt', vocab_size, special_tokens)
    with open(f'{root}{name}bpe_data.pkl', 'wb') as f:
        pickle.dump((vocab, merges, special_tokens), f)

In [9]:
def read_docs(file_path, num_docs, seperator):
    lines = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            index = line.find(seperator)
            if index != -1:
                num_docs -= 1
                if num_docs <= 0:
                    lines.append(line[:index])
                    break
                else:
                    lines.append(line)
            else:
                lines.append(line)
    return ''.join(lines)

tiny_story_docs = read_docs('../data/TinyStoriesV2-GPT4-train.txt', 10, '<|endoftext|>')
#print(tiny_story_docs)
owt_docs = read_docs('../data/owt_train.txt', 10, '<|endoftext|>')
tiny_story_bpe_info = pickle.load(open('../data/TinyStoriesV2-GPT4-bpe_data.pkl', 'rb'))
owt_bpe_info = pickle.load(open('../data/owt_bpe_data.pkl', 'rb'))
tiny_story_tokenizer = Tokenizer(*tiny_story_bpe_info)
owt_tokenizer = Tokenizer(*owt_bpe_info)

tiny_story_tokens = tiny_story_tokenizer.encode(tiny_story_docs)
owt_tokens = owt_tokenizer.encode(owt_docs)
print('tiny story compression ratio: ', len(tiny_story_docs) / len(tiny_story_tokens))
print('owt compression ratio: ', len(owt_docs) / len(owt_tokens))

owt_tokens_with_tiny_story_tokenizer = tiny_story_tokenizer.encode(owt_docs)
print('owt with tiny story tokenizer compression ratio: ', len(owt_docs) / len(owt_tokens_with_tiny_story_tokenizer))

tiny story compression ratio:  4.151898734177215
owt compression ratio:  4.655557208748698
owt with tiny story tokenizer compression ratio:  3.166363084395871
